In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm, spearmanr, kendalltau
from scipy.interpolate import griddata 
from scipy import interpolate
import sys

import lmom as lmom
from datetime import datetime
# import openturns as ot
# import openturns.viewer as viewer
# ot.Log.Show(ot.Log.NONE)

In [ ]:
c = 68
#Without A&V
# op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\05_HYDzoom" + r"\C%d_" %c + r"disc1d_"
#With A&V
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2" + r"\C%d_" %c + r"disc1d_"

AM1 = pd.read_pickle(op + "AM1.pkl")
AM2 = pd.read_pickle(op + "AM2.pkl")
AM3 = pd.read_pickle(op + "AM3.pkl")

The response functions

In [ ]:
# TSpath = r'\TS_S%d' %rp1 + r'_S%d.csv' %rp2 #  according to Q1_Q2
fn = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\Response_functions' 

sv = r'\Res_Area.csv'

Res_Area = pd.read_csv(fn + sv, delimiter=';')
sv = r'\Res_Volume.csv'
Res_Volume = pd.read_csv(fn + sv, delimiter=';')
sv = r'\Res_mDepth.csv'
Res_mDepth = pd.read_csv(fn + sv, delimiter=';')


Area = Res_Area.iloc[:,1:].values
Volume = Res_Volume.iloc[:,1:].values
mDepth = Res_mDepth.iloc[:,1:].values
# Area


S1 = [ 723.95,  3403.62,  6083.29,  8762.96, 11442.62]# np.array([  723.95,  3403.62,  6083.29,  8762.96, 11442.62]) #Colum
S2 = [ 27.14, 1119.63, 2212.12, 3304.61, 4397.12]# np.array([  27.14, 1119.63, 2212.12, 3304.61, 4397.12]) #INDEX
S1_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #Colum
S2_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #INDEX


#Interpolation grids
x = S1
y = S2
z = Area
fArea = interpolate.interp2d(x, y, z, kind='linear')

z = Volume
fVolume = interpolate.interp2d(x, y, z, kind='linear')

z = mDepth
fmDepth = interpolate.interp2d(x, y, z, kind='linear')

In [ ]:
x, y = np.meshgrid(S1, S2)
z = Res_Area.iloc[:,1:].values / 1000000
cmap = 'YlOrRd'
unit = 'm3/s'

plt.figure(figsize=(6, 5)) 
plt.contourf(x, y, z, cmap=cmap)
# plt.colorbar()
cbar = plt.colorbar(label="Area of inundation (km2)", orientation="vertical") #, fontsize = 16)
cbar.ax.tick_params(labelsize=12)
# cbar.ax.set_label(size='large')
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Response Function\nArea of inundation', fontsize = 20) 
plt.xlabel(f'S1 - Rhine River ({unit})', fontsize=16)
plt.ylabel(f'S2 - Main River ({unit})', fontsize=16)
# plt.grid()    
# plt.show()


x, y = np.meshgrid(S1, S2)
z = Res_Volume.iloc[:,1:].values / 1000000000

cmap = 'YlOrRd'
unit = 'm3/s'

plt.figure(figsize=(6, 5)) 
plt.contourf(x, y, z, cmap=cmap)
# plt.colorbar()
cbar = plt.colorbar(label="Volume of inundation (km3)", orientation="vertical") #, fontsize = 16)
cbar.ax.tick_params(labelsize=12)
# cbar.ax.set_label(size='large')
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Response Function\nVolume of inundation', fontsize = 20) 
plt.xlabel(f'S1 - Rhine River ({unit})', fontsize=16)
plt.ylabel(f'S2 - Main River ({unit})', fontsize=16)
# plt.grid()    
# plt.show()

### Benchmark
Return Periods
T=2, 5, 10, 20, 50, 100, 200, 500, 1000

In [ ]:
def ecdf(data):
    """ Compute Empirical CDF """
    x = np.sort(data)
    n = x.size
    rank = np.arange(1, n+1)
    y = rank / (n+1)
    return x, y

def empinv(yp, xed, yed):
    xp = []
    for i in range(len(yed)):
        if yp < yed[0]:
            xp=xed[0]
            break
        elif yp > yed[len(yed)-1]:
            xp=(xed[len(yed)-1])
            break
        elif yp == yed[i-1]:
            xp=(xed[i-1])
        elif yed[i-1] < yp < yed[i]:
            y1 = yed[i-1]
            y2 = yed[i]
            x1 = xed[i-1]
            x2 = xed[i]
            xcal = x1 + ((x2-x1)/(y2-y1)) * (yp-y1)
            xp=(xcal)
        #else:
         #   xp.append(xed[len(yed)-1])
    return xp

In [ ]:
# S1_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #Colum
# S2_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #INDEX
# Res_Area = pd.DataFrame(index=[S2_n] ,columns=[S1_n])
T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]
CV=0.05
N = [((1/CV)**2)/(1-p) for p in Pexc]
col = ['T', 'Pexc', 'N', 'A_Set1', 'A_Set2', 'A_Set3', 'V_Set1', 'V_Set2', 'V_Set3']
Tvalues = pd.DataFrame(columns=[col])
Tvalues['T'] = np.float64(T)
Tvalues['Pexc'] = np.float64(Pexc)
Tvalues['N'] = np.float64(N)

x, y = ecdf(AM1['Area'])
A_Set1 = [empinv(p, x, y)/1000000 for p in Pexc]
Tvalues['A_Set1'] = np.float64(A_Set1)

x, y = ecdf(AM2['Area'])
A_Set2 = [empinv(p, x, y)/1000000 for p in Pexc]
Tvalues['A_Set2'] = A_Set2

x, y = ecdf(AM3['Area'])
A_Set3 = [empinv(p, x, y)/1000000 for p in Pexc]
Tvalues['A_Set3'] = A_Set3

x, y = ecdf(AM1['Volume'])
V_Set1 = [empinv(p, x, y)/1000000000 for p in Pexc]
Tvalues['V_Set1'] = V_Set1

x, y = ecdf(AM2['Volume'])
V_Set2 = [empinv(p, x, y)/1000000000 for p in Pexc]
Tvalues['V_Set2'] = V_Set2

x, y = ecdf(AM3['Volume'])
V_Set3 = [empinv(p, x, y)/1000000000 for p in Pexc]
Tvalues['V_Set3'] = V_Set3

Tvalues

In [ ]:
sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Tvalues.pkl"
Tvalues.to_pickle(sv)

In [ ]:
x, y = np.meshgrid(S1, S2)
z = Res_Area.iloc[:,1:].values / 1000000
cmap = 'YlOrRd'
unit = 'm3/s'


A_levels = np.float64(A_Set1)

plt.figure(figsize=(6, 5)) 
CS = plt.contour(x, y, z, levels=A_levels, cmap=cmap)

fmt = {}
for l, t in zip(CS.levels, T):
    fmt[l] = f'T={t} years'

plt.clabel(CS, inline=True, fontsize=10, colors='k', fmt=fmt)
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Return Periods\nArea of inundation | Set1', fontsize = 20) 
plt.xlabel(f'S1 - Rhine River ({unit})', fontsize=16)
plt.ylabel(f'S2 - Main River ({unit})', fontsize=16)

A_levels = np.float64(A_Set2)

plt.figure(figsize=(6, 5)) 
CS = plt.contour(x, y, z, levels=A_levels, cmap=cmap)

fmt = {}
for l, t in zip(CS.levels, T):
    fmt[l] = f'T={t} years'

plt.clabel(CS, inline=True, fontsize=10, colors='k', fmt=fmt)
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Return Periods\nArea of inundation | Set2', fontsize = 20) 
plt.xlabel(f'S1 - Rhine River ({unit})', fontsize=16)
plt.ylabel(f'S2 - Main River ({unit})', fontsize=16)

A_levels = np.float64(A_Set3)

plt.figure(figsize=(6, 5)) 
CS = plt.contour(x, y, z, levels=A_levels, cmap=cmap)

fmt = {}
for l, t in zip(CS.levels, T):
    fmt[l] = f'T={t} years'

plt.clabel(CS, inline=True, fontsize=10, colors='k', fmt=fmt)
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Return Periods\nArea of inundation | Set3', fontsize = 20) 
plt.xlabel(f'S1 - Rhine River ({unit})', fontsize=16)
plt.ylabel(f'S2 - Main River ({unit})', fontsize=16)


In [ ]:

x, y = np.meshgrid(S1, S2)
z = Res_Volume.iloc[:,1:].values / 1000000000
cmap = 'YlOrRd'
unit = 'm3/s'


V_levels = np.float64(V_Set1)

plt.figure(figsize=(6, 5)) 
CS = plt.contour(x, y, z, levels=V_levels, cmap=cmap)

fmt = {}
for l, t in zip(CS.levels, T):
    fmt[l] = f'T={t} years'

plt.clabel(CS, inline=True, fontsize=10, colors='k', fmt=fmt)
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Return Periods\nVolume of inundation | Set1', fontsize = 20) 
plt.xlabel(f'S1 - Rhine River ({unit})', fontsize=16)
plt.ylabel(f'S2 - Main River ({unit})', fontsize=16)

V_levels = np.float64(V_Set2)

plt.figure(figsize=(6, 5)) 
CS = plt.contour(x, y, z, levels=V_levels, cmap=cmap)

fmt = {}
for l, t in zip(CS.levels, T):
    fmt[l] = f'T={t} years'

plt.clabel(CS, inline=True, fontsize=10, colors='k', fmt=fmt)
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Return Periods\nVolume of inundation | Set2', fontsize = 20) 
plt.xlabel(f'S1 - Rhine River ({unit})', fontsize=16)
plt.ylabel(f'S2 - Main River ({unit})', fontsize=16)

V_levels = np.float64(V_Set3)

plt.figure(figsize=(6, 5)) 
CS = plt.contour(x, y, z, levels=V_levels, cmap=cmap)

fmt = {}
for l, t in zip(CS.levels, T):
    fmt[l] = f'T={t} years'

plt.clabel(CS, inline=True, fontsize=10, colors='k', fmt=fmt)
plt.xticks(fontsize = 12, rotation = 90)
plt.yticks(fontsize = 12)
plt.title(f'Return Periods\nVolume of inundation | Set3', fontsize = 20) 
plt.xlabel(f'S1 - Rhine River ({unit})', fontsize=16)
plt.ylabel(f'S2 - Main River ({unit})', fontsize=16)

## Estimation of the design flood event: Copulas + Monte Carlo Sampling (+Bootstrapping experiment)

T-area and T-volume

Full data..

Marginals: Empiricals

In [ ]:
def GauR(AMset, n):
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])
    fp1, xp1 = ecdf(AMset['Q_S1'])
    fp2, xp2 = ecdf(AMset['Q_S2'])
    
    Area = np.zeros(n)
    Volume = np.zeros(n)
    for i in range(n):
        p1 = np.random.rand(1)
        p2 = np.random.rand(1)
        u1 = norm.ppf(p1)
        u2 = norm.ppf(p2)
        u1_ = u1
        u2_ = rho * u1 + np.sqrt(1 - rho**2) * u2
        p1_ = norm.cdf(u1_)
        p2_ = norm.cdf(u2_)    
        D1 = np.interp(p1_, xp1, fp1)
        D2 = np.interp(p2_, xp2, fp2) 
        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]
        
    return Area, Volume


def GumR(AMset, n):
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])
    fp1, xp1 = ecdf(AMset['Q_S1'])
    fp2, xp2 = ecdf(AMset['Q_S2'])
    
    Area = np.zeros(n)
    Volume = np.zeros(n)
        
    alpha = 1 / (1-tau)
    
    #lookup table
    x1 = np.arange(0.0000000001, 0.000000001, 0.0000000001)
    x2 = x1 * 10
    x3 = x2 * 10
    x4 = x3 * 10
    x5 = x4 * 10
    x6 = x5 * 10
    x7 = x6 * 10
    x8 = x7 * 10
    x9 = np.arange(x8[-1]+0.001, 0.999, 0.001)
    x10 = np.arange(0.999, 0.9999, 0.0001)
    x11 = np.arange(0.9999, 0.99999, 0.00001)
    x12 = np.arange(0.99999, 1, 0.000001)
    x = np.concatenate((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12))
    y = alpha * ((-np.log(x))** alpha)/(x*np.log(x))
    ltable = pd.DataFrame({'x':x, 'y':y})

    for i in range(n):
        u = np.random.rand(1)
        t = np.random.rand(1)
        phi_u = alpha * ((-np.log(u))**alpha)/(u*np.log(u))
        phi_w = phi_u / t
        
        w = np.interp(phi_w, ltable['y'], ltable['x'])
        s = ((-np.log(w)) ** alpha ) - ((-np.log(u)) ** alpha )
        v = np.exp(-(s**(1/alpha)))

        D1 = np.interp(u, xp1, fp1)
        D2 = np.interp(v, xp2, fp2)

        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]

    return Area, Volume
    

def ClaR(AMset, n):
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])
    fp1, xp1 = ecdf(AMset['Q_S1'])
    fp2, xp2 = ecdf(AMset['Q_S2'])
    Area = np.zeros(n)
    Volume = np.zeros(n)
    alpha = (2 * tau) / (1-tau)
    
    #lookup table
    x1 = np.arange(0.0000000001, 0.000000001, 0.0000000001)
    x2 = x1 * 10
    x3 = x2 * 10
    x4 = x3 * 10
    x5 = x4 * 10
    x6 = x5 * 10
    x7 = x6 * 10
    x8 = x7 * 10
    x9 = np.arange(x8[-1]+0.001, 0.999, 0.001)
    x10 = np.arange(0.999, 0.9999, 0.0001)
    x11 = np.arange(0.9999, 0.99999, 0.00001)
    x12 = np.arange(0.99999, 1, 0.000001)
    x = np.concatenate((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12))
    y = - ( x ** ( -1 - alpha))
    ltable = pd.DataFrame({'x':x, 'y':y})
    
    for i in range(n):
        u = np.random.rand(1)
        t = np.random.rand(1)
        phi_u =  -(u ** (-1 - alpha))
        phi_w = phi_u / t
        
        w = np.interp(phi_w, ltable['y'], ltable['x'])
        s = ((w ** (-alpha))-1)/alpha-((u ** (-alpha))-1)/alpha
        v = (alpha * s +1) ** (-1/alpha)

        D1 = np.interp(u, xp1, fp1)
        D2 = np.interp(v, xp2, fp2)

        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]

    return Area, Volume

In [ ]:
Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM2', 'AM3'] #'AM1', 
n = 200000   # CV=5% for T=500

Nsam = 50000

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now().time())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} at {str(datetime.now().time())}')
        Area, Volume = GauR(AMset, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR(AMset, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum


        Area, Volume = ClaR(AMset, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')


Samples 'N' size..


N= 20, 50, 100, 500, 1000
Marginals: Metrics fit (alternative 1)

In [ ]:
def GauR_n(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])

    #Marginals
    LMU1 = lmom.samlmu(AMset['Q_S1'])
    LMU2 = lmom.samlmu(AMset['Q_S2'])

    if AM == 'AM1':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)  
    elif AM == 'AM2':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgev(LMU2)
    elif AM == 'AM3':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)

    Area = np.zeros(n)
    Volume = np.zeros(n)
    for i in range(n): 
        p1 = np.random.rand(1)
        p2 = np.random.rand(1)
        u1 = norm.ppf(p1)
        u2 = norm.ppf(p2)
        u1_ = u1
        u2_ = rho * u1 + np.sqrt(1 - rho**2) * u2
        p1_ = norm.cdf(u1_)
        p2_ = norm.cdf(u2_)   

        if AM == 'AM1':
            D1 = lmom.quagev(p1_, fit1)
            D2 = lmom.quagpa(p2_, fit2)
        elif AM == 'AM2':
            D1 = lmom.quagev(p1_, fit1)
            D2 = lmom.quagev(p2_, fit2)
            
        elif AM == 'AM3':
            D1 = lmom.quagev(p1_, fit1)
            D2 = lmom.quagpa(p2_, fit2)


        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]
        
    return Area, Volume


def GumR_n(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])

    #Marginals
    LMU1 = lmom.samlmu(AMset['Q_S1'])
    LMU2 = lmom.samlmu(AMset['Q_S2'])

    if AM == 'AM1':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)  
    elif AM == 'AM2':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgev(LMU2)
        
    elif AM == 'AM3':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)

    Area = np.zeros(n)
    Volume = np.zeros(n)

    alpha = 1 / (1-tau)
    
    #lookup table
    x1 = np.arange(0.0000000001, 0.000000001, 0.0000000001)
    x2 = x1 * 10
    x3 = x2 * 10
    x4 = x3 * 10
    x5 = x4 * 10
    x6 = x5 * 10
    x7 = x6 * 10
    x8 = x7 * 10
    x9 = np.arange(x8[-1]+0.001, 0.999, 0.001)
    x10 = np.arange(0.999, 0.9999, 0.0001)
    x11 = np.arange(0.9999, 0.99999, 0.00001)
    x12 = np.arange(0.99999, 1, 0.000001)
    x = np.concatenate((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12))
    y = alpha * ((-np.log(x))** alpha)/(x*np.log(x))
    ltable = pd.DataFrame({'x':x, 'y':y})

    for i in range(n):
        u = np.random.rand(1)
        t = np.random.rand(1)
        phi_u = alpha * ((-np.log(u))**alpha)/(u*np.log(u))
        phi_w = phi_u / t
        
        w = np.interp(phi_w, ltable['y'], ltable['x'])
        s = ((-np.log(w)) ** alpha ) - ((-np.log(u)) ** alpha )
        v = np.exp(-(s**(1/alpha)))
        
        if AM == 'AM1':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagpa(v, fit2)
        elif AM == 'AM2':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagev(v, fit2) 
        elif AM == 'AM3':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagpa(v, fit2)

        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]

    return Area, Volume
    

def ClaR_n(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])
    
    #Marginals
    LMU1 = lmom.samlmu(AMset['Q_S1'])
    LMU2 = lmom.samlmu(AMset['Q_S2'])

    if AM == 'AM1':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)  
    elif AM == 'AM2':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgev(LMU2)
    elif AM == 'AM3':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)
        
    Area = np.zeros(n)
    Volume = np.zeros(n)
    
    alpha = (2 * tau) / (1-tau)
    
    #lookup table
    x1 = np.arange(0.0000000001, 0.000000001, 0.0000000001)
    x2 = x1 * 10
    x3 = x2 * 10
    x4 = x3 * 10
    x5 = x4 * 10
    x6 = x5 * 10
    x7 = x6 * 10
    x8 = x7 * 10
    x9 = np.arange(x8[-1]+0.001, 0.999, 0.001)
    x10 = np.arange(0.999, 0.9999, 0.0001)
    x11 = np.arange(0.9999, 0.99999, 0.00001)
    x12 = np.arange(0.99999, 1, 0.000001)
    x = np.concatenate((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12))
    y = - ( x ** ( -1 - alpha))
    ltable = pd.DataFrame({'x':x, 'y':y})

    # for i in range(n):
    iter = 0
    while iter<n:
        u = np.random.rand(1)
        t = np.random.rand(1)
        phi_u =  -(u ** (-1 - alpha))
        phi_w = phi_u / t
        
        w = np.interp(phi_w, ltable['y'], ltable['x'])
        s = ((w ** (-alpha))-1)/alpha-((u ** (-alpha))-1)/alpha
        v = (alpha * s +1) ** (-1/alpha)

        if AM == 'AM1':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagpa(v, fit2)
        elif AM == 'AM2':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagev(v, fit2)
        elif AM == 'AM3':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagpa(v, fit2)

        if D1 != None:
            if D2 != None:
                Area[iter] = fArea(D1, D2)[0]
                Volume[iter] = fVolume(D1, D2)[0]
                iter+=1
        else:
            iter+=0
            
    return Area, Volume

N=[20, 50, 100, 500, 1000]
Nsam = CHANGE

In [ ]:

Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM1'] #'AM1', 'AM2', 'AM3'
n = 200000   # CV=5% for T=500 MCiterations

Nsam = 20 #[20, 50, 100, 500, 1000] Size of the random sample

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now().time())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} - {str(datetime.now().time())}')
        Area, Volume = GauR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum

        Area, Volume = ClaR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')


In [ ]:

Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM1'] #'AM1', 'AM2', 'AM3'
n = 200000   # CV=5% for T=500 MCiterations

Nsam = 50 #[20, 50, 100, 500, 1000] Size of the random sample

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} - {str(datetime.now().time())}')
        Area, Volume = GauR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum

        Area, Volume = ClaR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')

In [ ]:
Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM1'] #'AM1', 'AM2', 'AM3'
n = 200000   # CV=5% for T=500 MCiterations

Nsam = 100 #[20, 50, 100, 500, 1000] Size of the random sample

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} - {str(datetime.now().time())}')
        Area, Volume = GauR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum

        Area, Volume = ClaR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')

In [ ]:

Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM1'] #'AM1', 'AM2', 'AM3'
n = 200000   # CV=5% for T=500 MCiterations

Nsam = 500 #[20, 50, 100, 500, 1000] Size of the random sample

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} - {str(datetime.now().time())}')
        Area, Volume = GauR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum

        Area, Volume = ClaR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')

In [ ]:
Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM1'] #, 'AM1', 'AM3'
n = 200000   # CV=5% for T=500 MCiterations

Nsam = 1000 #[20, 50, 100, 500, 1000] Size of the random sample

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} - {str(datetime.now().time())}')
        Area, Volume = GauR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum


        Area, Volume = ClaR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')

In [ ]:

Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM1', 'AM2', 'AM3'] #, 'AM2', 'AM3'
n = 200000   # CV=5% for T=500 MCiterations

Nsam = 5000 #[20, 50, 100, 500, 1000] Size of the random sample

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} - {str(datetime.now().time())}')
        Area, Volume = GauR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum


        Area, Volume = ClaR_n(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')

## Sensitivity analysis for the selection of the marginal distribution

Marginals accoding to log plots

In [ ]:
def GauR_n2(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])

    #Marginals
    LMU1 = lmom.samlmu(AMset['Q_S1'])
    LMU2 = lmom.samlmu(AMset['Q_S2'])

    if AM == 'AM1':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)  
    elif AM == 'AM2':
        fit1 = lmom.pelgum(LMU1)
        fit2 = lmom.pelgum(LMU2)
    elif AM == 'AM3':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgum(LMU2)

    Area = np.zeros(n)
    Volume = np.zeros(n)
    for i in range(n): 
        p1 = np.random.rand(1)
        p2 = np.random.rand(1)
        u1 = norm.ppf(p1)
        u2 = norm.ppf(p2)
        u1_ = u1
        u2_ = rho * u1 + np.sqrt(1 - rho**2) * u2
        p1_ = norm.cdf(u1_)
        p2_ = norm.cdf(u2_)   

        if AM == 'AM1':
            D1 = lmom.quagev(p1_, fit1)
            D2 = lmom.quagpa(p2_, fit2)
        elif AM == 'AM2':
            D1 = lmom.quagum(p1_, fit1)
            D2 = lmom.quagum(p2_, fit2)
            
        elif AM == 'AM3':
            D1 = lmom.quagev(p1_, fit1)
            D2 = lmom.quagum(p2_, fit2)


        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]
        
    return Area, Volume


def GumR_n2(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])

    #Marginals
    LMU1 = lmom.samlmu(AMset['Q_S1'])
    LMU2 = lmom.samlmu(AMset['Q_S2'])

    if AM == 'AM1':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)  
    elif AM == 'AM2':
        fit1 = lmom.pelgum(LMU1)
        fit2 = lmom.pelgum(LMU2)
        
    elif AM == 'AM3':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgum(LMU2)

    Area = np.zeros(n)
    Volume = np.zeros(n)

    alpha = 1 / (1-tau)
    
    #lookup table
    x1 = np.arange(0.0000000001, 0.000000001, 0.0000000001)
    x2 = x1 * 10
    x3 = x2 * 10
    x4 = x3 * 10
    x5 = x4 * 10
    x6 = x5 * 10
    x7 = x6 * 10
    x8 = x7 * 10
    x9 = np.arange(x8[-1]+0.001, 0.999, 0.001)
    x10 = np.arange(0.999, 0.9999, 0.0001)
    x11 = np.arange(0.9999, 0.99999, 0.00001)
    x12 = np.arange(0.99999, 1, 0.000001)
    x = np.concatenate((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12))
    y = alpha * ((-np.log(x))** alpha)/(x*np.log(x))
    ltable = pd.DataFrame({'x':x, 'y':y})

    for i in range(n):
        u = np.random.rand(1)
        t = np.random.rand(1)
        phi_u = alpha * ((-np.log(u))**alpha)/(u*np.log(u))
        phi_w = phi_u / t
        
        w = np.interp(phi_w, ltable['y'], ltable['x'])
        s = ((-np.log(w)) ** alpha ) - ((-np.log(u)) ** alpha )
        v = np.exp(-(s**(1/alpha)))
        
        if AM == 'AM1':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagpa(v, fit2)
        elif AM == 'AM2':
            D1 = lmom.quagum(u, fit1)
            D2 = lmom.quagum(v, fit2) 
        elif AM == 'AM3':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagum(v, fit2)

        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]

    return Area, Volume
    

def ClaR_n2(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])
    
    #Marginals
    LMU1 = lmom.samlmu(AMset['Q_S1'])
    LMU2 = lmom.samlmu(AMset['Q_S2'])

    if AM == 'AM1':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgpa(LMU2)  
    elif AM == 'AM2':
        fit1 = lmom.pelgum(LMU1)
        fit2 = lmom.pelgum(LMU2)
    elif AM == 'AM3':
        fit1 = lmom.pelgev(LMU1)
        fit2 = lmom.pelgum(LMU2)
        
    Area = np.zeros(n)
    Volume = np.zeros(n)
    
    alpha = (2 * tau) / (1-tau)
    
    #lookup table
    x1 = np.arange(0.0000000001, 0.000000001, 0.0000000001)
    x2 = x1 * 10
    x3 = x2 * 10
    x4 = x3 * 10
    x5 = x4 * 10
    x6 = x5 * 10
    x7 = x6 * 10
    x8 = x7 * 10
    x9 = np.arange(x8[-1]+0.001, 0.999, 0.001)
    x10 = np.arange(0.999, 0.9999, 0.0001)
    x11 = np.arange(0.9999, 0.99999, 0.00001)
    x12 = np.arange(0.99999, 1, 0.000001)
    x = np.concatenate((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12))
    y = - ( x ** ( -1 - alpha))
    ltable = pd.DataFrame({'x':x, 'y':y})

    # for i in range(n):
    iter = 0
    while iter<n:
        u = np.random.rand(1)
        t = np.random.rand(1)
        phi_u =  -(u ** (-1 - alpha))
        phi_w = phi_u / t
        
        w = np.interp(phi_w, ltable['y'], ltable['x'])
        s = ((w ** (-alpha))-1)/alpha-((u ** (-alpha))-1)/alpha
        v = (alpha * s +1) ** (-1/alpha)

        if AM == 'AM1':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagpa(v, fit2)
        elif AM == 'AM2':
            D1 = lmom.quagum(u, fit1)
            D2 = lmom.quagum(v, fit2)
        elif AM == 'AM3':
            D1 = lmom.quagev(u, fit1)
            D2 = lmom.quagum(v, fit2)

        if D1 != None:
            if D2 != None:
                Area[iter] = fArea(D1, D2)[0]
                Volume[iter] = fVolume(D1, D2)[0]
                iter+=1
        else:
            iter+=0
            
    return Area, Volume

2nd alternative for marginals SET 2 & SET3

In [ ]:
Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM2', 'AM3'] #'AM1', 'AM2', 'AM3'
n = 200000   # CV=5% for T=500 MCiterations

Nsam = 100 #[20, 50, 100, 500, 1000] Size of the random sample

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} - {str(datetime.now().time())}')
        Area, Volume = GauR_n2(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR_n2(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum

        Area, Volume = ClaR_n2(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau2.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau2.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum2.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum2.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla2.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla2.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')

In [ ]:
#Empirical distribution

def GauR_n3emp(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])

    #Marginals
    fp1, xp1 = ecdf(AMs['Q_S1'])
    fp2, xp2 = ecdf(AMs['Q_S2'])

    Area = np.zeros(n)
    Volume = np.zeros(n)
    for i in range(n): 
        p1 = np.random.rand(1)
        p2 = np.random.rand(1)
        u1 = norm.ppf(p1)
        u2 = norm.ppf(p2)
        u1_ = u1
        u2_ = rho * u1 + np.sqrt(1 - rho**2) * u2
        p1_ = norm.cdf(u1_)
        p2_ = norm.cdf(u2_)   

        D1 = np.interp(p1_, xp1, fp1)
        D2 = np.interp(p2_, xp2, fp2) 
       
        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]
        
    return Area, Volume

def GumR_n3emp(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])

    #Marginals
    fp1, xp1 = ecdf(AMs['Q_S1'])
    fp2, xp2 = ecdf(AMs['Q_S2'])

    Area = np.zeros(n)
    Volume = np.zeros(n)

    alpha = 1 / (1-tau)
    
    #lookup table
    x1 = np.arange(0.0000000001, 0.000000001, 0.0000000001)
    x2 = x1 * 10
    x3 = x2 * 10
    x4 = x3 * 10
    x5 = x4 * 10
    x6 = x5 * 10
    x7 = x6 * 10
    x8 = x7 * 10
    x9 = np.arange(x8[-1]+0.001, 0.999, 0.001)
    x10 = np.arange(0.999, 0.9999, 0.0001)
    x11 = np.arange(0.9999, 0.99999, 0.00001)
    x12 = np.arange(0.99999, 1, 0.000001)
    x = np.concatenate((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12))
    y = alpha * ((-np.log(x))** alpha)/(x*np.log(x))
    ltable = pd.DataFrame({'x':x, 'y':y})

    for i in range(n):
        u = np.random.rand(1)
        t = np.random.rand(1)
        phi_u = alpha * ((-np.log(u))**alpha)/(u*np.log(u))
        phi_w = phi_u / t
        
        w = np.interp(phi_w, ltable['y'], ltable['x'])
        s = ((-np.log(w)) ** alpha ) - ((-np.log(u)) ** alpha )
        v = np.exp(-(s**(1/alpha)))
        
        D1 = np.interp(u, xp1, fp1)
        D2 = np.interp(v, xp2, fp2)

        Area[i] = fArea(D1, D2)[0]
        Volume[i] = fVolume(D1, D2)[0]

    return Area, Volume
    

def ClaR_n3emp(AM, AMs, Nsam, n):
    AMs['ind'] = np.arange(len(AMs))
    randomindex1 = np.random.choice(AMs.ind, size=Nsam, replace=False)
    AMset = pd.DataFrame()
    for i in range(Nsam):
        # AMset = AMset.append(AMs.loc[AMs.ind == randomindex1[i]])
        AMset = pd.concat([AMset, AMs.loc[AMs.ind == randomindex1[i]]])
    
    rho, Sp = spearmanr(AMset['Q_S1'], AMset['Q_S2'])
    tau, Kp = kendalltau(AMset['Q_S1'], AMset['Q_S2'])
    
    #Marginals
    fp1, xp1 = ecdf(AMs['Q_S1'])
    fp2, xp2 = ecdf(AMs['Q_S2'])
        
    Area = np.zeros(n)
    Volume = np.zeros(n)
    
    alpha = (2 * tau) / (1-tau)
    
    #lookup table
    x1 = np.arange(0.0000000001, 0.000000001, 0.0000000001)
    x2 = x1 * 10
    x3 = x2 * 10
    x4 = x3 * 10
    x5 = x4 * 10
    x6 = x5 * 10
    x7 = x6 * 10
    x8 = x7 * 10
    x9 = np.arange(x8[-1]+0.001, 0.999, 0.001)
    x10 = np.arange(0.999, 0.9999, 0.0001)
    x11 = np.arange(0.9999, 0.99999, 0.00001)
    x12 = np.arange(0.99999, 1, 0.000001)
    x = np.concatenate((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12))
    y = - ( x ** ( -1 - alpha))
    ltable = pd.DataFrame({'x':x, 'y':y})

    # for i in range(n):
    iter = 0
    while iter<n:
        u = np.random.rand(1)
        t = np.random.rand(1)
        phi_u =  -(u ** (-1 - alpha))
        phi_w = phi_u / t
        
        w = np.interp(phi_w, ltable['y'], ltable['x'])
        s = ((w ** (-alpha))-1)/alpha-((u ** (-alpha))-1)/alpha
        v = (alpha * s +1) ** (-1/alpha)

        D1 = np.interp(u, xp1, fp1)
        D2 = np.interp(v, xp2, fp2)

        if D1 != None:
            if D2 != None:
                Area[iter] = fArea(D1, D2)[0]
                Volume[iter] = fVolume(D1, D2)[0]
                iter+=1
        else:
            iter+=0
            
    return Area, Volume   



In [ ]:
Nb = 100 #Bootstrapping experiment

T = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 5000]
Pexc = [1 - 1 / t for t in T]

AMs = ['AM1', 'AM2', 'AM3'] #'AM1', 'AM2', 'AM3'
n = 200000   # CV=5% for T=500 MCiterations

Nsam = 100 #[20, 50, 100, 500, 1000] Size of the random sample

for AM in AMs:
    print(f'Starting {AM} at {str(datetime.now())}')
    if AM == 'AM1':
        AMset = AM1 
    elif AM == 'AM2':
        AMset = AM2
    elif AM == 'AM3':
        AMset = AM1 
    
    globals()[f'N{Nsam}_{AM}_Agau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgau'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Agum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vgum'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Acla'] = pd.DataFrame(index=range(Nb), columns=T)
    globals()[f'N{Nsam}_{AM}_Vcla'] = pd.DataFrame(index=range(Nb), columns=T)
    
    for i in range(Nb):
        print(f'{i} - {str(datetime.now().time())}')
        Area, Volume = GauR_n3emp(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gau = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gau = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agau'].loc[i] = A_gau
        globals()[f'N{Nsam}_{AM}_Vgau'].loc[i] = V_gau

        Area, Volume = GumR_n3emp(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_gum = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_gum = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Agum'].loc[i] = A_gum
        globals()[f'N{Nsam}_{AM}_Vgum'].loc[i] = V_gum

        Area, Volume = ClaR_n3emp(AM, AMset, Nsam, n)
        x, y = ecdf(Area)
        A_cla = [empinv(p, x, y) for p in Pexc] #/1000000
        x, y = ecdf(Volume)
        V_cla = [empinv(p, x, y) for p in Pexc] #/1000000000
        globals()[f'N{Nsam}_{AM}_Acla'].loc[i] = A_cla
        globals()[f'N{Nsam}_{AM}_Vcla'].loc[i] = V_cla
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ2\MCruns\Ts\N%d" %Nsam + r"_" + AM
    desc = r"_Agau3emp.pkl"
    globals()[f'N{Nsam}_{AM}_Agau'].to_pickle(sv+desc)
    desc = r"_Vgau3emp.pkl"
    globals()[f'N{Nsam}_{AM}_Vgau'].to_pickle(sv+desc)
    desc = r"_Agum3emp.pkl"
    globals()[f'N{Nsam}_{AM}_Agum'].to_pickle(sv+desc)
    desc = r"_Vgum3emp.pkl"
    globals()[f'N{Nsam}_{AM}_Vgum'].to_pickle(sv+desc)
    desc = r"_Acla3emp.pkl"
    globals()[f'N{Nsam}_{AM}_Acla'].to_pickle(sv+desc)
    desc = r"_Vcla3emp.pkl"
    globals()[f'N{Nsam}_{AM}_Vcla'].to_pickle(sv+desc)
    
    print(f'Done {AM} at {str(datetime.now().time())}')